<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial-IllinoisGRMHD: NRPyfied IllinoisGRMHD Expressions

## Authors: Leo Werneck & Zach Etienne

<font color='red'>**This module is currently under development**</font>

## In this tutorial notebook we implement the NRPyfied version of expressions used in `IllinoisGRMHD`

### Required and recommended citations:

* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

0. [Step 0](#loading_modules): **Loading necessary Python/NRPy+ modules**
1. [Step 1](#adm_3metric_files): **`ADM_3METRIC` files**
    1. [Step 1.a](#compute_gamma_inv_and_sqrtgamma): *The `compute__ADM_gammaUU_and_sqrtgamma__in_terms_of__ADM_3METRIC.h` file*
    1. [Step 1.b](#compute_g4dd): *The `compute__g4DD__in_terms_of__ADM_3METRIC` file*
    1. [Step 1.c](#compute_g4uu): *The `compute__g4UU__in_terms_of__ADM_3METRIC` file*
1. [Step 2](#conf_metric_vars): **`CONF_METRIC` files**
    1. [Step 2.a](#read_in_conf_metric_vars): *The `VARS_FOR_METRIC_FACEVALS.h` and  `read_in_CONF_METRIC_from_gridfunctions.h` files*
    1. [Step 2.b](#read_in_conf_metric_vars): *The `compute__ADM_gammaDD__in_terms_of__CONF_METRIC.h`*
1. [Step 3](#tmunu): **Computing the energy-momentum tensor**
    1. [Step 3.a](#tmunu_grhd): *Computing the GRHD energy-momentum tensor*
    1. [Step 3.b](#tmunu_em): *Computing the EM energy-momentum tensor*
    1. [Step 3.c](#tmunu_tupmunu): *The `compute__TUPMUNU.h` file*
    1. [Step 3.d](#tmunu_tdnmunu): *The `compute__TDNMUNU.h` file*
1. [Step 4](#read_in_grmhd_vars): **The `GRMHD_VARS.h` and  `read_IN_PRIMS_and_OUT_PRIMS_from_gridfunctions.h` files**
1. [Step 5](#read_in_INTERP_VARS): **The `INTERP_VARS.h` and  `read_in_INTERP_VARS_from_gridfunctions.h` files**
1. [Step 6](#smallb_and_conservs_vars): **The `SMALLB_VARS.h` and  `CONSERV_VARS.h` files**
1. [Step 7](#impose_speed_limit_output_u0): **The `impose_speed_limit_output_u0.h` file**
1. [Step 8](#compute_tau_rhs_extrinsic_curvature_terms_and_tupmunu): **The `compute_tau_rhs_extrinsic_curvature_terms_and_TUPmunu.h` file**
1. [Step 9](#mhdflux): **GRMHD flux terms**
    1. [Step 9.a](#prim_to_conserv): *Computing conservatives from primitives*
    1. [Step 9.b](#compute_cp_and_cm): *Computing $c_{+}$ and $c_{-}$*
    1. [Step 9.c](#compute_cmax_and_cmin): *Computing $c_{\max}$ and $c_{\min}$*
    1. [Step 9.d](#speed_limit_and_u4u): *Impose speed limit and compute $u^{\mu}$*
    1. [Step 9.e](#computing_fhll): *Computing $F^{\rm HLL}$*
    1. [Step 9.f](#grmhd_flux): *Computing the GRMHD flux*
    1. [Step 9.g](#flux_dirn_i__h): *The `flux_dirn_x.h`, `flux_dirn_y.h`, and `flux_dirn_z.h` files*
1. [Step n](#latex_pdf_output): **Output this notebook to $\LaTeX$-formatted PDF file**

<a id='loading_modules'></a>

# Step 0: Loading necessary Python/NRPy+ modules \[Back to [top](#toc)\]
$$\label{loading_modules}$$

We now load the necessary Python and NRPy+ modules needed by this tutorial notebook.

In [1]:
# Import Python modules
import os,sys                # Python module: used for system and OS specific commands
import sympy as sp           # Python module: used for symbolic expressions
import re                    # Python module: used to manipulate regular expressions

# Register NRPy+ root directory to the path
nrpy_dir_path = os.path.join("..","..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

# Load NRPy+ modules
from outputC import *        # NRPy+ module: used to output sympy expressions to C
import indexedexp as ixp     # NRPy+ module: used to generate indexed expressions (e.g. g_{\mu\nu})
import cmdline_helper as cmd # NRPy+ module: used for command line features

# Create the NRPy+ header file directory, if it doesn't already exist
IGM_src_dir_path = os.path.join("..","src")
cmd.mkdir(os.path.join(IGM_src_dir_path,"NRPy_generated_headers"))
NRPy_headers_dir_path = os.path.join(IGM_src_dir_path,"NRPy_generated_headers")

# Set up a neat function to output the expressions to NRPy+ generated files
def NRPy_IGM_write_to_file(filepath,filename,contents):
    with open(filepath,"w") as file:
        file.write("""
/* .-----------------------------------------------------------------------.
 * | This file was generated by NRPy+ for IllinoisGRMHD, as documented in: |
 * |        Tutorial-IllinoisGRMHD__NRPyfied_IGM_expressions.ipynb         |
 * .-----------------------------------------------------------------------.
 * |                Author(s): Leo Werneck and Zach Etienne                |
 * .-----------------------------------------------------------------------.
 * |             Source: https://github.com/leowerneck/NRPyIGM             |
 * .-----------------------------------------------------------------------.
 *
 * File start: """+filename+""" */
\n"""+contents+"""
/* File end  : """+filename+""" */
""")
    print("Just generated the file: "+filepath)

# Set up a "variable definition" file generating function
def generate_variable_definition_file(gfslist,filename,varnameidx=0,comment="",extra=""):

    # Find the largest name inside the gfslist
    largest_name_len = len(gfslist[0][0])
    for j in range(len(gfslist)):
        if(len(gfslist[j][0]) > largest_name_len):
            largest_name_len =  len(gfslist[j][0])

    # Initialize string to comment
    string = comment
    for j in range(len(gfslist)):
        string += "static const int "+gfslist[j][varnameidx]
        for k in range(largest_name_len - len(gfslist[j][varnameidx])):
            string += " "
        
        string += " = "+str(j)+";\n"
    
    if(extra != ""):
        string += "\n"+extra
        
    # Write string to file
    filepath = os.path.join(NRPy_headers_dir_path,filename)
    NRPy_IGM_write_to_file(filepath,filename,string)

# Set up a "read from gf" file generating function
def generate_read_in_file(arrayname,gfslist,filename,varnameidx=0,gfnameidx=1,comment="",extra=""):
    
    # Find the largest name inside the gfslist
    largest_name_len = len(gfslist[0][0])
    for j in range(len(gfslist)):
        if(len(gfslist[j][0]) > largest_name_len):
            largest_name_len =  len(gfslist[j][0])
    
    # Initialize string to comment
    string = comment
    for j in range(len(gfslist)):
        string += arrayname+"["+gfslist[j][varnameidx]+"]"
        for k in range(largest_name_len - len(gfslist[j][varnameidx])):
            string += " "
        
        string += " = "+gfslist[j][gfnameidx]+";\n"
        
    if(extra != ""):
        string += "\n"+extra

    # Write string to file
    filepath = os.path.join(NRPy_headers_dir_path,filename)
    NRPy_IGM_write_to_file(filepath,filename,string)

<a id='adm_3metric_vars'></a>

# Step 1: The `ADM_3METRIC` variable declaration \[Back to [top](#toc)\]
$$\label{adm_3metric_vars}$$

The `ADM_3METRIC` array contains the following quantities:

\begin{align}
{\rm ALPHA} &:= \alpha\ ,\\
{\rm BETAU} &:= \beta^{i}\ ,\\
{\rm GDD}   &:= \gamma_{ij}\ ,\\
{\rm GUPDD} &:= \gamma^{ij}\ ,\\
{\rm SQRTGAMMA} &:= \sqrt{\gamma} \ ,\ \gamma \equiv \det\left(\gamma_{ij}\right)\ .
\end{align}

We start by declaring a function that sets up these variables.

In [2]:
# Step 1: Declare basic ADM variables to be used by IllinoisGRMHD
# Step 1.a: Set spatial dimension to 3
DIM = 3

# Step 1.b: Set up alpha
alpha  = sp.symbols("ADM_3METRIC[ALPHA]",real=True)
gfslist = [["alpha"]]
# Step 1.b: Set up beta^{i}
betaU   = ixp.zerorank1()
for i in range(DIM):
    betaU[i] = sp.symbols('ADM_3METRIC[BETA'+chr(ord('X')+i)+"]",real=True)
    gfslist.append(["BETA"+chr(ord('X')+i)])

# Step 1.c: Set up gamma_{ij}
gammaDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(i,DIM):
        gammaDD[i][j] = gammaDD[j][i] = sp.symbols('ADM_3METRIC[GAMMA'+chr(ord('X')+i)+chr(ord('X')+j)+"]",real=True)
        gfslist.append(["GAMMA"+chr(ord('X')+i)+chr(ord('X')+j)])

# Step 1.d: Set up gamma^{ij}
gammaUU = ixp.zerorank2()
for i in range(3):
    for j in range(i,3):
        gammaUU[i][j] = gammaUU[j][i] = sp.symbols('ADM_3METRIC[GAMMA'+chr(ord('X')+i)+chr(ord('X')+j)+"]",real=True)
        gfslist.append(["GAMMAUP"+chr(ord('X')+i)+chr(ord('X')+j)])
        
# Step 1.e: Set up sqrt(gamma)
sqrtgamma = sp.symbols("ADM_3METRIC[SQRTGAMMA]",real=True)
gfslist.append(["SQRTGAMMA"])

# Step 1.f: Define the indices values
# b^{\mu} quantities

# Set up the b^{\mu} string
comment = "/* ADM_3METRIC variables */\n"
filename = "ADM_3METRIC_VARS.h"
generate_variable_definition_file(gfslist,filename,comment=comment)

Just generated the file: ../src/NRPy_generated_headers/ADM_3METRIC_VARS.h


<a id='compute_gamma_inv_and_sqrtgamma'></a>

## Step 1.a: The `compute__ADM_gammaUU_and_sqrtgamma __in_terms_of__ADM_3METRIC.h` file \[Back to [top](#toc)\]
$$\label{compute_gamma_inv_and_sqrtgamma}$$

Now we compute the inverse ADM 3-metric $\gamma^{ij}$ from $\gamma_{ij}$, by inverting $\gamma_{ij}$ using our `symm_matrix_inverter3x3()` function from the [indexedexp.py](/edit/NRPyIGM/indexedexp.py) NRPy+ module.

In [3]:
# Step 1.a: The compute_ADM_gammaUU_and_sqrtgamma.h file
# Step 1.a.i: Compute eh inverse ADM 3-metric and the determinant of the ADM 3-metric
gammaINV, gammaDET = ixp.symm_matrix_inverter3x3(gammaDD)

# Step 1.a.ii: Populate ADM_3METRIC[GAMMAUPYZ] and ADM_3METRIC[SQRTGAMMA]
#              with the results of our inverter
exprlist = [sp.sqrt(gammaDET)]
varslist = ["ADM_3METRIC[SQRTGAMMA]"]
for i in range(DIM):
    for j in range(i,DIM):
        exprlist.append(gammaINV[i][j])
        varslist.append("ADM_3METRIC[GAMMAUP"+chr(ord('X')+i)+chr(ord('X')+j)+"]")

string = outputC(exprlist,varslist,filename="returnstring", params="outCverbose=False")

# Step 1.a.iii: Replace pow(blah, 2) with (blah)*(blah) and double with CCTK_REAL
string2 = re.sub('pow\(([^,]+), 2\)', '(\\1)*(\\1)', string); string = string2
string2 = re.sub('double','CCTK_REAL',string); string = string2

# Step 1.a.iv: Output result to file
filename = "compute__ADM_gammaUU_and_sqrtgamma__in_terms_of__ADM_3METRIC.h"
filepath = os.path.join(NRPy_headers_dir_path,filename)
NRPy_IGM_write_to_file(filepath,filename,string)

Just generated the file: ../src/NRPy_generated_headers/compute__ADM_gammaUU_and_sqrtgamma__in_terms_of__ADM_3METRIC.h


<a id='compute_g4dd'></a>

## Step 1.b: The `compute__g4DD__in_terms_of__ADM_3METRIC.h` file \[Back to [top](#toc)\]
$$\label{compute_g4dd}$$

We now compute the ADM 4-metric, $g_{\mu\nu}$, given by (see equation 2.122 in [Baumgarte & Shapiro's Numerical Relativity](https://www.google.com/books/edition/Numerical_Relativity/dxU1OEinvRUC?hl=en&gbpv=0))

$$
g_{\mu\nu}
=
\begin{pmatrix}
-\alpha^{2} + \beta_{\ell}\beta^{\ell} & \beta_{i}\\
\beta_{j} & \gamma_{ij}
\end{pmatrix}\ .
$$

We do this with using the [BSSN/ADMBSSN_tofrom_4metric](/edit/NRPyIGM/BSSN/ADMBSSN_tofrom_4metric.py) NRPy+ module.

In [4]:
# Step 1.b: Compute the ADM 4-metric, g_{\mu\nu}
# Step 1.b.i: Load the BSSN.ADMBSSN_tofrom_4metric NRPy+ module
import BSSN.ADMBSSN_tofrom_4metric as AB4m

# Step 1.b.ii: Compute the g_{\mu\nu} in terms of our ADM variables
AB4m.g4DD_ito_BSSN_or_ADM("ADM",gammaDD=gammaDD,betaU=betaU,alpha=alpha)
g4DD = AB4m.g4DD

# Step 1.b.iii: Set up lists to store the expressions and output variables
exprlist = []
namelist = []
for mu in range(4):
    for nu in range(4):
        exprlist.append(g4DD[mu][nu])
        namelist.append("g4dn["+str(mu)+"]["+str(nu)+"]")
        
# Step 1.b.iv: Convert our results to C output
string = outputC(exprlist,namelist,"returnstring", params="outCverbose=False")
# Step 1.b.v: Replace pow(blah, 2) with (blah)*(blah) and double with CCTK_REAL
string2 = re.sub('pow\(([^,]+), 2\)', '(\\1)*(\\1)', string); string = string2
string2 = re.sub('double','CCTK_REAL',string); string = string2

# Step 1.b.vi: Output to file
filename = "compute__g4DD__in_terms_of__ADM_3METRIC.h"
filepath = os.path.join(NRPy_headers_dir_path,filename)
NRPy_IGM_write_to_file(filepath,filename,string)

Just generated the file: ../src/NRPy_generated_headers/compute__g4DD__in_terms_of__ADM_3METRIC.h


<a id='compute_g4uu'></a>

## Step 1.c: The `compute__g4UU__in_terms_of__ADM_3METRIC.h` file \[Back to [top](#toc)\]
$$\label{compute_g4uu}$$

We now compute the inverse ADM 4-metric, $g^{\mu\nu}$, given by (see equation 2.119 in [Baumgarte & Shapiro's Numerical Relativity](https://www.google.com/books/edition/Numerical_Relativity/dxU1OEinvRUC?hl=en&gbpv=0))

$$
g^{\mu\nu}
=
\begin{pmatrix}
-\alpha^{-2} & \alpha^{-2}\beta^{i}\\
\alpha^{-2}\beta^{j} & \gamma^{ij} - \alpha^{-2}\beta^i\beta^{j}
\end{pmatrix}\ .
$$

We do this with using the [BSSN/ADMBSSN_tofrom_4metric](/edit/NRPyIGM/BSSN/ADMBSSN_tofrom_4metric.py) NRPy+ module.

In [5]:
# Step 1.c: Compute the inverse ADM 4-metric, g^{\mu\nu}
# Step 1.c.i: Load the BSSN.ADMBSSN_tofrom_4metric NRPy+ module
import BSSN.ADMBSSN_tofrom_4metric as AB4m

# Step 1.c.ii: Compute the g_{\mu\nu} in terms of our ADM variables
AB4m.g4UU_ito_BSSN_or_ADM("ADM",betaU=betaU,alpha=alpha,gammaUU=gammaUU)
g4UU = AB4m.g4UU

# Step 1.c.iii: Set up lists to store the expressions and output variables
exprlist = []
namelist = []
for mu in range(4):
    for nu in range(4):
        exprlist.append(g4UU[mu][nu])
        namelist.append("g4up["+str(mu)+"]["+str(nu)+"]")
        
# Step 1.c.iv: Convert our results to C output
string = outputC(exprlist,namelist,"returnstring", params="outCverbose=False")
# Step 1.c.v: Replace pow(blah, 2) with (blah)*(blah) and double with CCTK_REAL
string2 = re.sub('pow\(([^,]+), 2\)', '(\\1)*(\\1)', string); string = string2
string2 = re.sub('double','CCTK_REAL',string); string = string2

# Step 1.c.vi: Output to file
filename = "compute__g4UU__in_terms_of__ADM_3METRIC.h"
filepath = os.path.join(NRPy_headers_dir_path,filename)
NRPy_IGM_write_to_file(filepath,filename,string)

Just generated the file: ../src/NRPy_generated_headers/compute__g4UU__in_terms_of__ADM_3METRIC.h


<a id='conf_metric_vars'></a>

# Step 2: The `CONF_METRIC` variable declaration \[Back to [top](#toc)\]
$$\label{conf_metric_vars}$$

The `CONF_METRIC` array contains the following quantities:

\begin{align}
{\rm CM\_PHI} &:= \phi\ ,\ \text{conformal factor}\ ,\\
{\rm CM\_PSI} &:= \psi \equiv e^{\phi}\ ,\\
{\rm CM\_GAMMADD} &:= \bar\gamma_{ij}\ ,\\
{\rm CM\_GAMMAUU} &:= \bar\gamma^{ij}\ .
\end{align}

We start by declaring a function that sets up these variables.

In [6]:
# Step 2: Set the the conformal metric variables to be used by IllinoisGRMHD
# Step 2.a: Set up phi
cf_phi = sp.symbols("CONF_METRIC[CM_PHI]",real=True)
cf_psi = sp.symbols("CONF_METRIC[CM_PSI]",real=True)

# Step 2.b: Set up \bar\gamma_{ij}
gammabarDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(i,DIM):
        gammabarDD[i][j] = gammabarDD[j][i] = sp.symbols('CONF_METRIC[CM_GAMMATILDE'+chr(ord('X')+i)+chr(ord('X')+j)+"]",real=True)

# Step 2.c: Set up \bar\gamma^{ij}
gammabarUU = ixp.zerorank2()
for i in range(3):
    for j in range(i,3):
        gammabarUU[i][j] = gammabarUU[j][i] = sp.symbols('CONF_METRIC[CM_GAMMATILDEUP'+chr(ord('X')+i)+chr(ord('X')+j)+"]",real=True)

<a id='read_in_conf_metric_vars'></a>

## Step 2.a: The `VARS_FOR_METRIC_FACEVALS.h` and  `read_in_CONF_METRIC_from_gridfunctions.h` files \[Back to [top](#toc)\]
$$\label{read_in_conf_metric_vars}$$

We now create a header files to substitute the following pieces of code:

1. In `IllinoisGRMHD_headers.h`:

```c
// The order here MATTERS, as we assume that GAMMAUPXX+1=GAMMAUPYY, etc.
static const int PHI=0,PSI=1,GAMMATILDEXX=2,GAMMATILDEXY=3,GAMMATILDEXZ=4,GAMMATILDEYY=5,GAMMATILDEYZ=6,GAMMATILDEZZ=7,
  LAPM1=8,SHIFTX=9,SHIFTY=10,SHIFTZ=11,GAMMATILDEUPXX=12,GAMMATILDEUPYY=13,GAMMATILDEUPZZ=14,
  NUMVARS_FOR_METRIC_FACEVALS=15; //<-- Be _sure_ to set this correctly, or you'll have memory access bugs!

// These are not used for facevals in the reconstruction step, but boy are they useful anyway. 
static const int GAMMAUPXY=15,GAMMAUPXZ=16,GAMMAUPYZ=17,
  NUMVARS_FOR_METRIC=18; //<-- Be _sure_ to set this correctly, or you'll have memory access bugs!
```
2. In `driver_evaluate_MHD_rhs.C`, `outer_boundaries.C`, `driver_conserv_to_prims.C`, and `set_IllinoisGRMHD_metric_GRMHD_variables_based_on_HydroBase_and_ADMBase_variables.C`, the last one being part of the `ID_converter_ILGRMHD` ETK thorn:

```c
  METRIC[ww]=phi_bssn;ww++;
  METRIC[ww]=psi_bssn;ww++;
  METRIC[ww]=gtxx;    ww++;
  METRIC[ww]=gtxy;    ww++;
  METRIC[ww]=gtxz;    ww++;
  METRIC[ww]=gtyy;    ww++;
  METRIC[ww]=gtyz;    ww++;
  METRIC[ww]=gtzz;    ww++;
  METRIC[ww]=lapm1;   ww++;
  METRIC[ww]=betax;   ww++;
  METRIC[ww]=betay;   ww++;
  METRIC[ww]=betaz;   ww++;
  METRIC[ww]=gtupxx;  ww++;
  METRIC[ww]=gtupyy;  ww++;
  METRIC[ww]=gtupzz;  ww++;
```

For starters, we will replace `METRIC` by `CONF_METRIC`.

In [7]:
# Set INDEXNAME and GFNAME
INDEXNAME = int(0)
GFNAME    = int(1)

# Start setting up the gridfunction indices with phi and psi
gfslist = [["CM_PHI","phi_bssn"],["CM_PSI","psi_bssn"]]

# Add the indices for \tilde{\gamma}_{ij}
for i in range(DIM):
    for j in range(i,DIM):
        gfslist.append(["CM_GAMMATILDE"+chr(ord('X')+i)+chr(ord('X')+j),"gt"+chr(ord('x')+i)+chr(ord('x')+j)])

# Add alpha, \beta^{i}, and \tilde{\gamma}^{ii}
gfslist.append(["CM_LAPM1","lapm1"])
for i in range(DIM):
    gfslist.append(["CM_SHIFT"+chr(ord('X')+i),"beta"+chr(ord('x')+i)])

for i in range(DIM):
    gfslist.append(["CM_GAMMATILDEUP"+chr(ord('X')+i)+chr(ord('X')+i),"gt"+chr(ord('x')+i)+chr(ord('x')+i)])

gfslist.append(["NUMVARS_FOR_CONF_METRIC_FACEVALS",""])
othervars =     [["CM_GAMMATILDEUPXY","gtxy"]]
othervars.append(["CM_GAMMATILDEUPXZ","gtxz"])
othervars.append(["CM_GAMMATILDEUPYZ","gtyz"])

# Set up extra code, not supported by the file generating function
extra = "/* Other useful variables */\n"
for j in range(len(othervars)):
    extra += "static const int "+othervars[j][INDEXNAME]
    for k in range(len("NUMVARS_FOR_CONF_METRIC_FACEVALS") - len(othervars[j][INDEXNAME])):
        extra += " "
    extra += " = "+str(j+len(gfslist)-1)+";\n"
extra += "static const int NUMVARS_FOR_CONF_METRIC          = "+str(len(gfslist)+len(othervars)-1)+";\n\n"

# Set up comments
comment = "/* Variables used for face value reconstructions */\n"
# Set up output file name
filename = "VARS_FOR_CONF_METRIC_FACEVALS.h"
# Generate variable definition file
generate_variable_definition_file(gfslist,filename,comment=comment,extra=extra)
    
# Set up the output array name
arrayname = "CONF_METRIC"
# Set up comments
comment   = "/* Reading in conformal metric face value variables from gridfunctions */\n"
# Set up output file name
filename  = "read_in_CONF_METRIC_FACEVALS_from_gridfunctions.h"
# Generate read in from gridfunctions file
generate_read_in_file(arrayname,gfslist[:-1],filename,comment=comment)

gfslist = gfslist[:-1]
gfslist.append(["CM_GAMMATILDEUPXY","gtxy"])
gfslist.append(["CM_GAMMATILDEUPXZ","gtxz"])
gfslist.append(["CM_GAMMATILDEUPYZ","gtyz"])
# Set up the output array name
arrayname = "CONF_METRIC"
# Set up comments
comment   = "/* Reading in conformal metric variables from gridfunctions */\n"
# Set up output file name
filename  = "read_in_CONF_METRIC_from_gridfunctions.h"
# Generate read in from gridfunctions file
generate_read_in_file(arrayname,gfslist,filename,comment=comment)

Just generated the file: ../src/NRPy_generated_headers/VARS_FOR_CONF_METRIC_FACEVALS.h
Just generated the file: ../src/NRPy_generated_headers/read_in_CONF_METRIC_FACEVALS_from_gridfunctions.h
Just generated the file: ../src/NRPy_generated_headers/read_in_CONF_METRIC_from_gridfunctions.h


<a id='read_in_conf_metric_vars'></a>

## Step 2.b: The `compute__ADM_gammaDD__in_terms_of__CONF_METRIC.h` \[Back to [top](#toc)\]
$$\label{read_in_conf_metric_vars}$$

Next we compute

$$
\gamma_{ij} = \psi^{4}\bar{\gamma}_{ij}
$$

In [8]:
gamma_ADM_ito_gammabar_DD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        gamma_ADM_ito_gammabar_DD[i][j] = cf_psi**4 * gammabarDD[i][j]
        
# Step 1.c: Set up gamma_{ij}
gammaDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(i,DIM):
        gammaDD[i][j] = gammaDD[j][i] = sp.symbols('ADM_3METRIC[GAMMA'+chr(ord('X')+i)+chr(ord('X')+j)+"]",real=True)

exprlist = []
varslist = []
for i in range(DIM):
    for j in range(DIM):
        exprlist.append(gamma_ADM_ito_gammabar_DD[i][j])
        varslist.append(gammaDD[i][j])
        
# Step 3.c.iv: Convert our results to C output
string = outputC(exprlist,varslist,"returnstring", params="outCverbose=False")

# Step 3.c.v: Replace pow(blah, 2) with (blah)*(blah) and double with CCTK_REAL
string2 = re.sub('pow\(([^,]+), 2\)', '(\\1)*(\\1)', string); string = string2
string2 = re.sub('double','CCTK_REAL',string); string = string

# Step 3.c.vi: Output to file
filename = "compute__ADM_gammaDD__in_terms_of__CONF_METRIC.h"
filepath = os.path.join(NRPy_headers_dir_path,filename)
NRPy_IGM_write_to_file(filepath,filename,string)

Just generated the file: ../src/NRPy_generated_headers/compute__ADM_gammaDD__in_terms_of__CONF_METRIC.h


<a id='tmunu'></a>

# Step 3: Computing the energy-momentum tensor \[Back to [top](#toc)\]
$$\label{tmunu}$$

The energy-momentum tensor for general relativistic magnetohydrodynamics (GRMHD) is given by

$$
T^{\mu\nu}_{\rm GRMHD} = T^{\mu\nu}_{\rm GRHD} + T^{\mu\nu}_{\rm EM}\ ,
$$

where

$$
T^{\mu\nu}_{\rm GRHD} = h\rho_{b}u^{\mu}u^{\nu} + Pg^{\mu\nu}\ ,
$$

is the general relativistic hydrodynamics (GRHD) energy-momentum tensor and 

$$
T^{\mu\nu}_{\rm EM} = b^2 u^{\mu} u^{\nu} + \frac{b^2}{2} g^{\mu\nu} + b^\mu b^\nu\ ,
$$

is the electromagnetic energy-momentum tensors, respectively. Here, $u^{\mu}$ is the fluid's 4-velocity and

\begin{align}
\sqrt{4\pi} b^0 = B^0_{\rm (u)} &= \frac{u_j B^j}{\alpha}\ ,\\
\sqrt{4\pi} b^i = B^i_{\rm (u)} &= \frac{B^i + (u_j B^j) u^i}{\alpha u^0}\ ,
\end{align}

with $b^{2}\equiv g_{\mu\nu}b^{\mu}b^{\nu}$.

<a id='tmunu_grhd'></a>

## Step 3.a: Computing the GRHD energy-momentum tensor \[Back to [top](#toc)\]
$$\label{tmunu_grhd}$$

$\newcommand{\Pcold}{P_{\text{cold}}}$
$\newcommand{\epscold}{\epsilon_{\text{cold}}}$
$\newcommand{\epsth}{\epsilon_{\text{th}}}$
$\newcommand{\Gammath}{\Gamma_{\text{th}}}$
$\newcommand{\rhob}{\rho_{b}}$

In order to compute the GRHD energy-momentum tensor, we will need the following quantities:

1. The enthalpy, $h$
1. The baryonic density, $\rho_{b}$
1. the 4-velocity, $u^{\mu}$
1. The pressure, $P$

The enthalpy is given by

$$
h = 1 + \epsilon + \frac{P}{\rhob}\ .
$$

For the hybrid EOS used by `IllinoisGRMHD`, namely

$$
P(\rho_{b},\epsilon) = \Pcold(\rhob) + \left(\Gammath-1\right)\rhob\left(\epsilon - \epscold\right)\ ,
$$

we have

$$
\boxed{\epsilon = \epscold + \frac{P-\Pcold}{\left(\Gammath-1\right)\rhob}}\ .
$$

To compute the 4-velocity $u^{\mu}$, we will use the `u4U_in_terms_of_vU_apply_speed_limit()` function from the [GRHD.equations NRPy+ module](/edit/NRPyIGM/GRHD/equations.py). Finally, the GRHD energy-momentum tensor is computed using the `compute_T4UU()` function, from the same module.

In [9]:
# Step 3: Computing T^{\mu\nu}_{GRMHD} and T_{\mu\nu}^{GRMHD}
# Step 3.a: Computing T^{\mu\nu}_{GRHD}
# Step 3.a.i: Import the GRHD NRPy+ module
import GRHD.equations as GRHD

# Step 3.a.ii: Declare needed variables, named according to IllinoisGRMHD
rho_b, P, P_cold, eps_cold, Gamma_th = sp.symbols("U[RHOB] U[PRESSURE] P_cold eps_cold Gamma_th",real=True)

# Step 3.a.iii: Compute epsilon
epsilon = eps_cold + (P - P_cold)/(rho_b * (Gamma_th - 1))

# Step 3.a.iv: Compute u^{\mu} from v^{i}
vU = [sp.symbols("U[VX]",real=True),sp.symbols("U[VY]",real=True),sp.symbols("U[VZ]",real=True)]
GRHD.u4U_in_terms_of_vU_apply_speed_limit(alpha,betaU,gammaDD, vU)
u4U = GRHD.u4_ito_3velsU

# Step 3.a.v: Compute the energy-momentum tensor
GRHD.compute_T4UU(gammaDD,betaU,alpha, rho_b,P,epsilon,u4U)

<a id='tmunu_em'></a>

## Step 3.b: Computing the EM energy-momentum tensor \[Back to [top](#toc)\]
$$\label{tmunu_em}$$

In order to compute the EM energy-momentum tensor, we will need to compute $b^{\mu}$ and $b^{2}$. We then compute the EM energy-momentum tensor using the `compute_TEM4UU()` from the [GRFFE.equations NRPy+ module](/edit/NRPyIGM/GRFFE/equations.py)

In [10]:
# Step 3.b: Computing T^{\mu\nu}_{EM}
# Step 3.b.i: Import the GRFFE NRPy+ module
import GRFFE.equations as GRFFE

# Step 3.b.ii: Declare necessary variables, named according to IllinoisGRMHD
smallb4U = [sp.symbols("smallb[SMALLBT]",real=True), \
            sp.symbols("smallb[SMALLBX]",real=True), \
            sp.symbols("smallb[SMALLBY]",real=True), \
            sp.symbols("smallb[SMALLBZ]",real=True)]

# Step 3.b.iii: Compute b^{2} = g_{\mu\nu}b^{\mu}b^{\nu}
smallbsquared = sp.sympify(0)
for mu in range(4):
    for nu in range(4):
        smallbsquared += g4DD[mu][nu] * smallb4U[mu] * smallb4U[nu]

# Step 3.b.iv: Compute the EM energy-momentum tensor
GRFFE.compute_TEM4UU(gammaDD,betaU,alpha, smallb4U, smallbsquared,u4U)

<a id='tmunu_tupmunu'></a>

## Step 3.c: The `compute__TUPMUNU.h` file \[Back to [top](#toc)\]
$$\label{tmunu_tupmunu}$$

Now that we have $T^{\mu\nu}_{\rm GRHD}$ and $T^{\mu\nu}_{\rm EM}$, we can easily compute the GRMHD energy-momentum tensor

$$
\boxed{T^{\mu\nu}_{\rm GRMHD} = T^{\mu\nu}_{\rm GRHD} + T^{\mu\nu}_{\rm EM}}\ .
$$

In [11]:
# Step 3.c: The compute_TUPMUNU.h file
# Step 3.c.i: Compute T^{\mu\nu}_{GRMHD}
def compute_GRMHD_T4UU(alpha,betaU,gammaDD, rho_b,P,epsilon,u4U, smallb4U,smallbsquared):
    global T4grmhdUU
    
    import GRHD.equations as GRHD
    GRHD.compute_T4UU(gammaDD,betaU,alpha, rho_b,P,epsilon,u4U)
    
    import GRFFE.equations as GRFFE
    GRFFE.compute_TEM4UU(gammaDD,betaU,alpha, smallb4U, smallbsquared,u4U)
    
    T4grmhdUU = GRHD.T4UU + GRFFE.TEM4UU
    
    return

# Step 3.c.ii: Prepare T^{\mu\nu}_{GRMHD} for C code output
compute_GRMHD_T4UU(alpha,betaU,gammaDD, rho_b,P,epsilon,u4U, smallb4U,smallbsquared)
T4igmUU = []
for mu in range(4):
    for nu in range(4):
        T4igmUU.append(T4grmhdUU[mu][nu])

# Step 3.c.iii: Declare the variables expected by IllinoisGRMHD
TUPMUNU = ["TUPMUNU["+str(count)+"]" for count in range(16)]

# Step 3.c.iv: Convert our results to C output
string = outputC(T4igmUU,TUPMUNU,"returnstring", params="outCverbose=False")

# Step 3.c.v: Replace pow(blah, 2) with (blah)*(blah) and double with CCTK_REAL
string2 = re.sub('pow\(([^,]+), 2\)', '(\\1)*(\\1)', string); string = string2
string2 = re.sub('double','CCTK_REAL',string); string = string

# Step 3.c.vi: Output to file
filename = "compute__TUPMUNU.h"
filepath = os.path.join(NRPy_headers_dir_path,filename)
NRPy_IGM_write_to_file(filepath,filename,string)

Just generated the file: ../src/NRPy_generated_headers/compute__TUPMUNU.h


<a id='tmunu_tdnmunu'></a>

## Step 3.d: The `compute__TDNMUNU.h` file \[Back to [top](#toc)\]
$$\label{tmunu_tdnmunu}$$

Now we compute 

$$
\boxed{T_{\mu\nu}^{\rm GRMHD} = g_{\mu\rho}g_{\nu\sigma}T^{\rho\sigma}_{\rm GRMHD}}\ .
$$

In [12]:
# Step 3.d: The compute_TDNMUNU.h file
# Step 3.d.i: Compute T_{\mu\nu}^{GRMHD}
def compute_GRMHD_T4DD(alpha,betaU,gammaDD, rho_b,P,epsilon,u4U, smallb4U,smallbsquared):
    global T4grmhdDD
    compute_GRMHD_T4UU(alpha,betaU,gammaDD, rho_b,P,epsilon,u4U, smallb4U,smallbsquared)
    T4grmhdDD = ixp.zerorank2(DIM=4)
    for mu in range(4):
        for nu in range(4):
            for rho in range(4):
                for sigma in range(4):
                    T4grmhdDD[mu][nu] += g4DD[mu][rho] * g4DD[nu][sigma] * T4grmhdUU[rho][sigma]
                    
    return

# Step 3.d.ii: Prepare T^{\mu\nu}_{GRMHD} for C code output
compute_GRMHD_T4DD(alpha,betaU,gammaDD, rho_b,P,epsilon,u4U, smallb4U,smallbsquared)
T4igmDD = []
for mu in range(4):
    for nu in range(4):
        T4igmDD.append(T4grmhdDD[mu][nu])

# Step 3.d.iii: Declare the variables expected by IllinoisGRMHD
TDNMUNU = ["TDNMUNU["+str(count)+"]" for count in range(16)]

# Step 3.d.iv: Convert our results to C output
string = outputC(T4igmDD,TDNMUNU,"returnstring", params="outCverbose=False")

# Step 3.d.v: Replace pow(blah, 2) with (blah)*(blah) and double with CCTK_REAL
string2 = re.sub('pow\(([^,]+), 2\)', '(\\1)*(\\1)', string); string = string2
string2 = re.sub('double','CCTK_REAL',string); string = string

# Step 3.d.vi: Output to file
filename = "compute__TDNMUNU.h"
filepath = os.path.join(NRPy_headers_dir_path,filename)
NRPy_IGM_write_to_file(filepath,filename,string)

Just generated the file: ../src/NRPy_generated_headers/compute__TDNMUNU.h


<a id='read_in_tupmunu'></a>

## Step 3.e: The `read_TUPMUNU_from_gridfunctions.h` files \[Back to [top](#toc)\]
$$\label{read_in_tupmunu}$$

We now write down a header file to substitute:

```c
  ww=0;
  TUPMUNU[ww]=TUPtt; ww++;
  TUPMUNU[ww]=TUPtx; ww++;
  TUPMUNU[ww]=TUPty; ww++;
  TUPMUNU[ww]=TUPtz; ww++;
  TUPMUNU[ww]=TUPxx; ww++;
  TUPMUNU[ww]=TUPxy; ww++;
  TUPMUNU[ww]=TUPxz; ww++;
  TUPMUNU[ww]=TUPyy; ww++;
  TUPMUNU[ww]=TUPyz; ww++;
  TUPMUNU[ww]=TUPzz; ww++;
```

In [13]:
# Set INDEXNAME, GFNAME, GFNAME_R, and GFNAME_L
INDEXNAME = int(0)
GFNAME    = int(1)

# Then, the right/left values of the 3-velocity
gfslist =     [["TMUNU_TT","TUPtt"]]
for i in range(DIM):
    gfslist.append(["TMUNU_T"+chr(ord('X')+i),"TUPt"+chr(ord('x')+i)])

for i in range(DIM):
    for j in range(i,DIM):
        gfslist.append(["TMUNU_"+chr(ord('X')+i)+chr(ord('X')+j),"TUP"+chr(ord('x')+i)+chr(ord('x')+j)])

# Set up comments
comment = "/* Define TMUNU variables (valid for all variants) */\n"
# Set up output file name
filename = "TMUNU_VARS.h"
# Generate variable definition file
generate_variable_definition_file(gfslist,filename,comment=comment)
    
# Set up the output array name
arrayname = "TUPMUNU"
# Set up comments
comment   = "/* Read in TUPMUNU variables from gridfunctions */\n"
# Set up output file name
filename  = "read_in_TUPMUNU_from_gridfunctions"
# Generate read in from gridfunctions file
generate_read_in_file(arrayname,gfslist,filename,comment=comment)

Just generated the file: ../src/NRPy_generated_headers/TMUNU_VARS.h
Just generated the file: ../src/NRPy_generated_headers/read_in_TUPMUNU_from_gridfunctions


<a id='read_in_grmhd_vars'></a>

# Step 4: The `GRMHD_VARS.h` and  `read_IN_PRIMS_and_OUT_PRIMS_from_gridfunctions.h` files \[Back to [top](#toc)\]
$$\label{read_in_grmhd_vars}$$

We now write codes to substitute:

1. In `IllinoisGRMHD_headers.h`:
```c
// The order here MATTERS, and must be consistent with the order in the IN_PRIMS[] array in driver_evaluate_MHD_rhs.C.
static const int RHOB=0,PRESSURE=1,VX=2,VY=3,VZ=4,
  BX_CENTER=5,BY_CENTER=6,BZ_CENTER=7,BX_STAGGER=8,BY_STAGGER=9,BZ_STAGGER=10,
  VXR=11,VYR=12,VZR=13,VXL=14,VYL=15,VZL=16,MAXNUMVARS=17;  //<-- Be _sure_ to define MAXNUMVARS appropriately!
static const int UT=0,UX=1,UY=2,UZ=3;
```

2. In `driver_evaluate_MHD_rhs.C`:
```c
int ww=0;
IN_PRIMS[ww].gf=rho_b;      OUT_PRIMS_R[ww].gf=rho_br;      OUT_PRIMS_L[ww].gf=rho_bl;      ww++;
IN_PRIMS[ww].gf=P;          OUT_PRIMS_R[ww].gf=Pr;          OUT_PRIMS_L[ww].gf=Pl;          ww++;
IN_PRIMS[ww].gf=vx;         OUT_PRIMS_R[ww].gf=vxr;         OUT_PRIMS_L[ww].gf=vxl;         ww++;
IN_PRIMS[ww].gf=vy;         OUT_PRIMS_R[ww].gf=vyr;         OUT_PRIMS_L[ww].gf=vyl;         ww++;
IN_PRIMS[ww].gf=vz;         OUT_PRIMS_R[ww].gf=vzr;         OUT_PRIMS_L[ww].gf=vzl;         ww++;
IN_PRIMS[ww].gf=Bx;         OUT_PRIMS_R[ww].gf=Bxr;         OUT_PRIMS_L[ww].gf=Bxl;         ww++;
IN_PRIMS[ww].gf=By;         OUT_PRIMS_R[ww].gf=Byr;         OUT_PRIMS_L[ww].gf=Byl;         ww++;
IN_PRIMS[ww].gf=Bz;         OUT_PRIMS_R[ww].gf=Bzr;         OUT_PRIMS_L[ww].gf=Bzl;         ww++;
IN_PRIMS[ww].gf=Bx_stagger; OUT_PRIMS_R[ww].gf=Bx_staggerr; OUT_PRIMS_L[ww].gf=Bx_staggerl; ww++;
IN_PRIMS[ww].gf=By_stagger; OUT_PRIMS_R[ww].gf=By_staggerr; OUT_PRIMS_L[ww].gf=By_staggerl; ww++;
IN_PRIMS[ww].gf=Bz_stagger; OUT_PRIMS_R[ww].gf=Bz_staggerr; OUT_PRIMS_L[ww].gf=Bz_staggerl; ww++;
IN_PRIMS[ww].gf=vxr;        OUT_PRIMS_R[ww].gf=vxrr;        OUT_PRIMS_L[ww].gf=vxrl;        ww++;
IN_PRIMS[ww].gf=vyr;        OUT_PRIMS_R[ww].gf=vyrr;        OUT_PRIMS_L[ww].gf=vyrl;        ww++;
IN_PRIMS[ww].gf=vzr;        OUT_PRIMS_R[ww].gf=vzrr;        OUT_PRIMS_L[ww].gf=vzrl;        ww++;
IN_PRIMS[ww].gf=vxl;        OUT_PRIMS_R[ww].gf=vxlr;        OUT_PRIMS_L[ww].gf=vxll;        ww++;
IN_PRIMS[ww].gf=vyl;        OUT_PRIMS_R[ww].gf=vylr;        OUT_PRIMS_L[ww].gf=vyll;        ww++;
IN_PRIMS[ww].gf=vzl;        OUT_PRIMS_R[ww].gf=vzlr;        OUT_PRIMS_L[ww].gf=vzll;        ww++;
```

In [14]:
# Set INDEXNAME, GFNAME, GFNAME_R, and GFNAME_L
INDEXNAME = int(0)
GFNAME    = int(1)
GFNAME_R  = int(2)
GFNAME_L  = int(3)

# Add indices and variables
# First for rho_b and the pressure
gfslist =     [["RHOB","rho_b","rho_br","rho_bl"]]
gfslist.append(["PRESSURE","P","Pr","Pl"])

# Second, the 3-velocity
gfslist.append(["VX","vx","vxr","vxl"])
gfslist.append(["VY","vy","vyr","vyl"])
gfslist.append(["VZ","vz","vzr","vzl"])

# Next, the unstaggered and staggered magnetic field
gfslist.append(["BX_CENTER","Bx","Bxr","Bxl"])
gfslist.append(["BY_CENTER","By","Byr","Byl"])
gfslist.append(["BZ_CENTER","Bz","Bzr","Bzl"])
gfslist.append(["BX_STAGGER","Bx_stagger","Bx_staggerr","Bx_staggerl"])
gfslist.append(["BY_STAGGER","By_stagger","By_staggerr","By_staggerl"])
gfslist.append(["BZ_STAGGER","Bz_stagger","Bz_staggerr","Bz_staggerl"])

# Then, the right/left values of the 3-velocity
gfslist.append(["VXR","vxr","vxrr","vxrl"])
gfslist.append(["VYR","vyr","vyrr","vyrl"])
gfslist.append(["VZR","vzr","vzrr","vzrl"])
gfslist.append(["VXL","vxl","vxlr","vxll"])
gfslist.append(["VYL","vyl","vylr","vyll"])
gfslist.append(["VZL","vzl","vzlr","vzll"])
gfslist.append(["MAXNUMVARS",""])

# Finally, the 4-velocity
u4list = ["UT","UX","UY","UZ"]

extra = "/* 4-velocity */\n"
for mu in range(4):
    extra += "static const int "+u4list[mu]
    for k in range(len("MAXNUMVARS")-len(u4list[mu])):
        extra += " "
    extra += " = "+str(mu)+";\n"

# Set up comments
comment = "/* GRMHD variables */\n"
# Set up output file name
filename = "GRMHD_VARS.h"
# Generate variable definition file
generate_variable_definition_file(gfslist,filename,comment=comment,extra=extra)
    
# Generate files to read in GRMHD quantities from gridfunctions
arraynames = ["IN_PRIMS","OUT_PRIMS_R","OUT_PRIMS_L"]
for j in range(len(arraynames)):
    comment     = "/* Reading in "+arraynames[j]+" */\n"
    filename    = "read_in_"+arraynames[j]+"_from_gridfunctions.h"
    generate_read_in_file(arraynames[j],gfslist[:-1],filename,comment=comment,gfnameidx=j+1)

Just generated the file: ../src/NRPy_generated_headers/GRMHD_VARS.h
Just generated the file: ../src/NRPy_generated_headers/read_in_IN_PRIMS_from_gridfunctions.h
Just generated the file: ../src/NRPy_generated_headers/read_in_OUT_PRIMS_R_from_gridfunctions.h
Just generated the file: ../src/NRPy_generated_headers/read_in_OUT_PRIMS_L_from_gridfunctions.h


<a id='read_in_INTERP_VARS'></a>

# Step 5: The `INTERP_VARS.h` and  `read_in_INTERP_VARS_from_gridfunctions.h` files \[Back to [top](#toc)\]
$$\label{read_in_INTERP_VARS}$$

1. In `IllinoisGRMHD_headers.h`:
```c
// The "I" suffix denotes interpolation. In other words, these
//    definitions are used for interpolation ONLY. The order here
//    matters as well!
static const int SHIFTXI=0,SHIFTYI=1,SHIFTZI=2,GAMMAUPXXI=3,GAMMAUPXYI=4,GAMMAUPXZI=5,GAMMAUPYYI=6,GAMMAUPYZI=7,GAMMAUPZZI=8,
  PSII=9,LAPM1I=10,A_XI=11,A_YI=12,A_ZI=13,LAPSE_PSI2I=14,LAPSE_OVER_PSI6I=15,MAXNUMINTERP=16;
```

2. In `driver_evaluate_MHD_rhs.C`:
```c
  ww=0;
  INTERP_VARS[ww]=betax;   ww++;
  INTERP_VARS[ww]=betay;   ww++;
  INTERP_VARS[ww]=betaz;   ww++;
  INTERP_VARS[ww]=gtupxx;  ww++;
  INTERP_VARS[ww]=gtupxy;  ww++;
  INTERP_VARS[ww]=gtupxz;  ww++;
  INTERP_VARS[ww]=gtupyy;  ww++;
  INTERP_VARS[ww]=gtupyz;  ww++;
  INTERP_VARS[ww]=gtupzz;  ww++;
  INTERP_VARS[ww]=psi_bssn;ww++;
  INTERP_VARS[ww]=lapm1;   ww++;
  INTERP_VARS[ww]=Ax;      ww++;
  INTERP_VARS[ww]=Ay;      ww++;
  INTERP_VARS[ww]=Az;      ww++;
  int max_num_interp_variables=ww;
```

In [15]:
# Set INDEXNAME and GFNAME
INDEXNAME = int(0)
GFNAME    = int(1)

# Shift vector, beta^{i}
gfslist =     [["INTERP_SHIFTX","betax"]]
gfslist.append(["INTERP_SHIFTY","betay"])
gfslist.append(["INTERP_SHIFTZ","betaz"])

# Conformal inverse metric, \tilde{\gamma}^{ij}
gfslist.append(["INTERP_GAMMATILDEUPXX","gtupxx"])
gfslist.append(["INTERP_GAMMATILDEUPXY","gtupxy"])
gfslist.append(["INTERP_GAMMATILDEUPXZ","gtupxz"])
gfslist.append(["INTERP_GAMMATILDEUPYY","gtupyy"])
gfslist.append(["INTERP_GAMMATILDEUPYZ","gtupyz"])
gfslist.append(["INTERP_GAMMATILDEUPZZ","gtupzz"])

# psi and alpha-1
gfslist.append(["INTERP_PSI","psi_bssn"])
gfslist.append(["INTERP_LAPM1","lapm1"])

# A fields
gfslist.append(["INTERP_AX","Ax"])
gfslist.append(["INTERP_AY","Ay"])
gfslist.append(["INTERP_AZ","Az"])

# Auxiliary quantities
gfslist.append(["INTERP_LAPSE_PSI2"])
gfslist.append(["INTERP_LAPSE_OVER_PSI6"])
gfslist.append(["MAXNUMINTERP",""])

# Define the indices in the GRMHD_VARS.h header file
# Start with the variables used in the face value reconstructions
comment = "/* Interpolation variables */\n"
filename = "INTERP_VARS.h"
generate_variable_definition_file(gfslist,filename,comment=comment)

extra = "int max_num_interp_variables"
for k in range(len("INTERP_VARS[INTERP_GAMMATILDEUPZZ]") - len("int max_num_interp_variables")):
    extra += " "
extra += " = "+str(len(gfslist[:-3]))+";\n"

comment = "/* Read in interpolation variables from gridfunctions */\n"
filename = "read_in_INTERP_VARS_from_gridfunctions.h"
arrayname = "INTERP_VARS"
generate_read_in_file(arrayname,gfslist[:-3],filename,comment=comment,extra=extra)

Just generated the file: ../src/NRPy_generated_headers/INTERP_VARS.h
Just generated the file: ../src/NRPy_generated_headers/read_in_INTERP_VARS_from_gridfunctions.h


<a id='smallb_and_conservs_vars'></a>

# Step 6: The `SMALLB_VARS.h` and  `CONSERV_VARS.h` files \[Back to [top](#toc)\]
$$\label{smallb_and_conservs_vars}$$

We now set up files to substitute the following lines in `IllinoisGRMHD_headers.h`:

```c
// Again, the order here MATTERS, since we assume in the code that, e.g., smallb[0]=b^t, smallb[3]=b^z, etc.
static const int SMALLBT=0,SMALLBX=1,SMALLBY=2,SMALLBZ=3,SMALLB2=4,NUMVARS_SMALLB=5;

// Again, the order here MATTERS, since we assume in the code that, CONSERV[STILDEX+1] = \tilde{S}_y
static const int RHOSTAR=0,STILDEX=1,STILDEY=2,STILDEZ=3,TAUENERGY=4,NUM_CONSERVS=5;
```

In [16]:
# b^{\mu} quantities
gfslist =     [["SMALLBT"]]
gfslist.append(["SMALLBX"])
gfslist.append(["SMALLBY"])
gfslist.append(["SMALLBZ"])
gfslist.append(["SMALLB2"])
gfslist.append(["NUMVARS_SMALLB"])

# Set up the b^{\mu} string
comment = "/* smallb (b^{\mu}) variables */\n"
filename = "SMALLB_VARS.h"
generate_variable_definition_file(gfslist,filename,comment=comment)

# Conservative quantities
gfslist =     [["RHOSTAR"]]
gfslist.append(["STILDEX"])
gfslist.append(["STILDEY"])
gfslist.append(["STILDEZ"])
gfslist.append(["TAUENERGY"])
gfslist.append(["NUM_CONSERVS"])

# Set up the b^{\mu} string
comment = "/* Interpolation variables */\n"
filename = "CONSERV_VARS.h"
generate_variable_definition_file(gfslist,filename,comment=comment)

Just generated the file: ../src/NRPy_generated_headers/SMALLB_VARS.h
Just generated the file: ../src/NRPy_generated_headers/CONSERV_VARS.h


<a id='impose_speed_limit_output_u0'></a>

# Step 7: The `impose_speed_limit_output_u0.h` file \[Back to [top](#toc)\]
$$\label{impose_speed_limit_output_u0}$$

We will not write down a header file to substitute the function `impose_speed_limit_output_u0()`, found in the [inlined_functions.C](Tutorial-IllinoisGRMHD__inlined_functions.ipynb) `IllinoisGRMHD` file. To this end, we will make use of the `compute_u0_noif()` function, developed for [GiRaFFE and implemented in this tutorial notebook](../../Tutorial-GiRaFFE_NRPy_Ccode_library-Stilde-flux.ipynb).

The `compute_u0_noif()` function imposes the speed limit on the *valencia* 3-velocity, $v^{i}_{(n)}$. However, in `IllinoisGRMHD` we use the 3-velocity

$$
v^{i} \equiv \frac{u^{i}}{u^{0}} = \alpha v^{i}_{(n)}-\beta^{i}\implies v^{i}_{(n)} = \frac{1}{\alpha}\left(v^{i}+\beta^{i}\right)\ .
$$

Thus, we first compute the Valencia 3-velocity from $v^{i}$, impose the speed limit to it, and then recompute $v^{i}_{\rm rescaled}$ from $\left(v^{i}_{(n)}\right)_{\rm rescaled}$, where by *rescaled* we really mean "multiplied by the speed limiting factor".

In [17]:
# Step 7: Impose speed limit and output u^{0}
# Step 7.a: Compute the Valencia 3-velocity: v^{i}_{(n)} = (v^{i} + beta^{i})/alpha
def rescale_3velocity_vU_and_compute_u0(alpha,betaU,gammaDD, vU):
    global rescaledvU, rescaledu0
    ValenciavU = ixp.zerorank1()
    for i in range(DIM):
        ValenciavU[i] = ( vU[i] + betaU[i] )/alpha

    # Step 7.b: Apply the speed limit to the Valencia 3-velocity and compute u^{0}
    import GiRaFFE_HO.Stilde_flux as stflux
    stflux.compute_u0_noif(gammaDD,alpha,ValenciavU)
    rescaledu0 = stflux.rescaledu0
    
    # Step 7.c: Recompute the 3-velocity v^{i} = alpha*v^{i}_{(n)} - beta^{i}
    rescaledvU = ixp.zerorank1()
    for i in range(DIM):
        rescaledvU[i] = alpha*stflux.rescaledValenciavU[i] - betaU[i]

# Step 7.d: Generate C string for v^{i} and u^{0}
rescale_3velocity_vU_and_compute_u0(alpha,betaU,gammaDD, vU)
explist = [rescaledvU[0], rescaledvU[1], rescaledvU[2], rescaledu0]
varlist = ["U[VX]","U[VY]","U[VZ]","u0_out"]
string = outputC(explist,varlist,filename="returnstring", params="outCverbose=False")

# Step 7.e: Replace pow(blah, 2) with (blah)*(blah) and double with CCTK_REAL
string2 = re.sub('pow\(([^,]+), 2\)', '(\\1)*(\\1)', string); string = string2
string2 = re.sub('double','CCTK_REAL',string); string = string2

# Step 7.f: Output to file
filename = "impose_speed_limit_output_u0.h"
filepath = os.path.join(NRPy_headers_dir_path,filename)
NRPy_IGM_write_to_file(filepath,filename,string)

Just generated the file: ../src/NRPy_generated_headers/impose_speed_limit_output_u0.h


<a id='compute_tau_rhs_extrinsic_curvature_terms_and_tupmunu'></a>

# Step 8: The `compute_tau_rhs_extrinsic_curvature_terms_and_TUPmunu.h` file \[Back to [top](#toc)\]
$$\label{compute_tau_rhs_extrinsic_curvature_terms_and_tupmunu}$$

$$
\partial_{t}\tilde\tau = \left[\rm Flux\ term\right] + s\ ,
$$

where

$$
s = \underbrace{\alpha\sqrt{\gamma}\left(T^{00}\beta^{i}\beta^{j} + 2T^{0i}\beta^{j} + T^{ij}\right)K_{ij}}_{\text{Extrinsic curvature terms}} - \underbrace{\left(T^{00}\beta^{i} + T^{0i}\right)\partial_{i}\alpha}_{\text{Will be implemented later}}\ .
$$

Here, we implement $s$. We consider that the inputs of the function are:

1. $\rm ADM\_3METRIC$: which gives us access to $\alpha$, $\beta^{i}$, $\gamma_{ij}$, and $\gamma^{ij}$
1. $\rm U$: which gives us access to $\rho_{b}$, $P$, $B^{i}$


In [18]:
def compute_tau_rhs_extrinsic_curvature_terms_and_TUPmunu(alpha,betaU,gammaDD,KDD, rho_b,P,epsilon,u4U, smallb4U,smallbsquared):
    
    # Compute extrinsic curvature terms of \partial_{t}\tilde\tau, namely:
    # .-------------------------------------------------------------------------------.
    # |     vvvvvvvvvvvvvvvvvvvvvvv  Extrinsic curvature term vvvvvvvvvvvvvvvvvvvvvvv |
    # | s = alpha*sqrt(gamma) ( T^{00}beta^{i}beta^{j} + 2T^{0i}beta^{j}T{ij} )K_{ij} |
    # .-------------------------------------------------------------------------------.
    
    # Start by computing T^{\mu\nu}
    compute_GRMHD_T4UU(alpha,betaU,gammaDD, rho_b,P,epsilon,u4U, smallb4U,smallbsquared)
    
    # Define s as a global variable and initialize it to zero
    global s
    s = sp.sympify(0)
    
    # Add extrinsic curvature term to s (modulo the multiplicative factor)
    for i in range(DIM):
        for j in range(DIM):
            s += (T4grmhdUU[0][0]*betaU[i]*betaU[j] + 2*T4grmhdUU[0][i]*betaU[j] + T4grmhdUU[i][j])*KDD[i][j]
            
    # Multiply s by alpha sqrt(gamma)
    s *= alpha*sqrtgamma
    
# Step 7.d: Generate C string for v^{i} and u^{0}
tau_rhs_alpha = sp.symbols("CONF_METRIC[CM_LAPM1]",real=True) + 1

tau_rhs_betaU = ixp.zerorank1()
for i in range(DIM):
    tau_rhs_betaU[i] = sp.symbols('CONF_METRIC[CM_SHIFT'+chr(ord('X')+i)+"]",real=True)

psi = sp.symbols("CONF_METRIC[CM_PSI]",real=True)

tau_rhs_gammaDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        tau_rhs_gammaDD[i][j] = psi**(-4) * gammabarDD[i][j]

tau_rhs_KDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(i,DIM):
        tau_rhs_KDD[i][j] = sp.symbols('k'+chr(ord('x')+i)+chr(ord('x')+j),real=True)
        tau_rhs_KDD[j][i] = sp.symbols('k'+chr(ord('x')+i)+chr(ord('x')+j),real=True)

smallb4U = [sp.symbols("smallb[SMALLBT]",real=True), \
            sp.symbols("smallb[SMALLBX]",real=True), \
            sp.symbols("smallb[SMALLBY]",real=True), \
            sp.symbols("smallb[SMALLBZ]",real=True)]


# Step 3.b.iii: Compute b^{2} = g_{\mu\nu}b^{\mu}b^{\nu}
AB4m.g4DD_ito_BSSN_or_ADM("ADM",gammaDD=tau_rhs_gammaDD,betaU=tau_rhs_betaU,alpha=tau_rhs_alpha)
tau_rhs_g4DD = AB4m.g4DD


smallbsquared = sp.sympify(0)
for mu in range(4):
    for nu in range(4):
        smallbsquared += tau_rhs_g4DD[mu][nu] * smallb4U[mu] * smallb4U[nu]

GRHD.u4U_in_terms_of_vU_apply_speed_limit(tau_rhs_alpha,tau_rhs_betaU,tau_rhs_gammaDD, vU)
tau_rhs_u4U = GRHD.u4_ito_3velsU
        
compute_tau_rhs_extrinsic_curvature_terms_and_TUPmunu(tau_rhs_alpha,tau_rhs_betaU,tau_rhs_gammaDD,tau_rhs_KDD,
                                                      rho_b,P,epsilon,tau_rhs_u4U, smallb4U,smallbsquared)
exprlist = []
varslist = []
for mu in range(4):
    for nu in range(4):
        exprlist.append(T4grmhdUU[mu][nu])
        varslist.append("TUPMUNU["+str(mu)+str(nu)+"]")
        
exprlist.append(s)
varslist.append("""
if(k<cctk_lsh[2]-cctk_nghostzones[2] && j<cctk_lsh[1]-cctk_nghostzones[1] && i<cctk_lsh[0]-cctk_nghostzones[0]) {

tau_rhs[index]""")

string = outputC(exprlist,varslist,filename="returnstring", params="outCverbose=False", poststring="\n   }\n\n")

# Step 7.e: Replace pow(blah, 2) with (blah)*(blah) and double with CCTK_REAL
string2 = re.sub('pow\(([^,]+), 2\)', '(\\1)*(\\1)', string); string = string2
string2 = re.sub('double','CCTK_REAL',string); string = string2

# Step 7.f: Output to file
filename = "compute_tau_rhs_extrinsic_curvature_terms_and_TUPmunu.h"
filepath = os.path.join(NRPy_headers_dir_path,filename)
NRPy_IGM_write_to_file(filepath,filename,string)

Just generated the file: ../src/NRPy_generated_headers/compute_tau_rhs_extrinsic_curvature_terms_and_TUPmunu.h


<a id='mhdflux'></a>

# Step 9: GRMHD flux terms \[Back to [top](#toc)\]
$$\label{mhdflux}$$

<a id='prim_to_conserv'></a>

## Step 9.a: Computing conservatives from primitives \[Back to [top](#toc)\]
$$\label{prim_to_conserv}$$

This is a fairly straightforward procedure, since:

$$
\boxed{
\begin{aligned}
\rho_{\star}  &= \alpha\sqrt{\gamma}\rho_{b}u^{0}\\
\tilde{\tau}  &= \alpha^{2}\sqrt{\gamma}T^{00} - \rho_{\star}\\
\tilde{S}_{i} &= \alpha\sqrt{\gamma}T^{0}_{\ \ i}
\end{aligned}
}\ .
$$

In [19]:
def primitives_to_conservatives(alpha,betaU,gammaDD,sqrtgamma, rhob,P,epsilon,u4U, smallb4U):
    
    # Compute g_{\mu\nu}
    AB4m.g4DD_ito_BSSN_or_ADM("ADM",gammaDD=gammaDD,betaU=betaU,alpha=alpha)
    g4DD = AB4m.g4DD

    # Compute b^{2} = g_{\mu\nu}b^{\mu}b^{\nu}
    smallbsquared = sp.sympify(0)
    for mu in range(4):
        for nu in range(4):
            smallbsquared += g4DD[mu][nu] * smallb4U[mu] * smallb4U[nu]

    # Start by computing T^{\mu\nu}
    compute_GRMHD_T4UU(alpha,betaU,gammaDD, rhob,P,epsilon,u4U, smallb4U,smallbsquared)
    T4grmhdUD = ixp.zerorank2(DIM=4)
    for mu in range(4):
        for nu in range(4):
            for sigma in range(4):
                T4grmhdUD[mu][nu] += g4DD[nu][sigma] * T4grmhdUU[sigma][mu]

    # Compute rho_star
    rho_star = alpha * sqrtgamma * rhob * u4U[0]

    # Compute tau
    tau = alpha**2 * sqrtgamma * T4grmhdUU[0][0] - rho_star

    # Compute Stilde
    StildeD = ixp.zerorank1()
    for i in range(DIM):
        StildeD[i] = alpha * sqrtgamma * T4grmhdUD[0][i+1]
        
    return rho_star, tau, StildeD

<a id='compute_cp_and_cm'></a>

## Step 9.b: Computing $c_{+}$ and $c_{-}$ \[Back to [top](#toc)\]
$$\label{compute_cp_and_cm}$$

In this function we will implement the algorithm discussed in the [inlined_functions.C tutorial notebook](Tutorial-IllinoisGRMHD__inlined_functions.ipynb), repeated here for the sake of the reader.

We approximate the general GRMHD dispersion relation (eq. 27 of [Gammie & McKinney (2003)](https://arxiv.org/pdf/astro-ph/0301509.pdf)) by the simpler expression

$$
\omega_{\rm cm}^{2} = \left[v_{\rm A}^{2} + c_{\rm s}^{2}\left(1-v_{\rm A}^{2}\right)\right]k_{\rm cm}^{2}\ ,
$$

where $\omega_{\rm cm}=-k_{\mu}u^{\mu}$ is the frequency and $k_{\rm cm}^{2} = K_{\mu}K^{\mu}$ the wavenumber of an MHD wave mode in the frame comoving with the fluid, where $K_{\mu}$ is defined as the projection of the wave vector $k^{\nu}$ onto the direction normal to $u^{\nu}$: $K_{\mu} = \left(g_{\mu\nu}+u_{\mu}u_{\nu}\right)k^{\nu}$. $c_{\rm s}$ is the sound speed, and $v_{\rm A}$ is the Alfvén speed, given by

$$
v_{\rm A} = \sqrt{\frac{b^{2}}{\rho_{b}h + b^{2}}}\ .
$$

With these definitions, we may then solve the approximate dispersion relation above along direction $i$, noting that in the comoving frame $k_{\mu} = \left(-\omega,k_{j}\delta^{j}_{\ i}\right)$ and the wave (phase) velocity is $c_{\pm} = \left.\omega\middle/\left(k_{j}\delta^{j}_{\ i}\right)\right.$. The dispersion can then be written as a quadratic equation for $c_{\pm}$:

$$
ac_{\pm}^{2} + bc_{\pm} + c = 0\ ,
$$

with

$$
\boxed{
\begin{align}
a &= \left(1-v_{0}^{2}\right)\left(u^{0}\right)^{2} - v_{0}^{2}g^{00}\ ,\\
b &= 2v_{0}^{2}g^{i0} - 2u^{i}u^{0}\left(1-v^{2}_{0}\right)\ ,\\
c &= \left(1-v_{0}^{2}\right)\left(u^{i}\right)^{2} - v_{0}^{2}g^{ii}\ ,\\
v_{0}^{2} &= v_{\rm A}^{2} + c_{\rm s}^{2}\left(1-v_{\rm A}^{2}\right)\ ,\\
c_{\rm s} &= \left.\left[\frac{dP_{\rm cold}}{d\rho_{b}} + \Gamma_{\rm th}\left(\Gamma_{\rm th}-1\right)\epsilon_{\rm th}\right]\middle/h\right.\ ,\\
c_{+} &= \max\left(\frac{-b \pm \sqrt{b^{2}-4ac}}{2a}\right)\ ,\\
c_{-} &= \min\left(\frac{-b \pm \sqrt{b^{2}-4ac}}{2a}\right)\ .
\end{align}
}
$$

In [20]:
def compute_v02( smallbsquared, rhob,dPcold_drho,h,Gamma_th,eps_th ):
    # Compute v_{A}^{2}
    vAlfvensquared = smallbsquared/( rhob*h + smallbsquared )
    
    # Compute c_{s}
    SoundSpeed = ( dPcold_drho + Gamma_th*(Gamma_th - 1)*eps_th ) / h
    
    # Compute v_{0}^{2}
    v0squared = vAlfvensquared + (1 - vAlfvensquared) * SoundSpeed**2
    
    return v0squared

def compute_cp_and_cm(flux_dirn, g4UU, smallbsquared, rhob,dPcold_drho,h,Gamma_th,eps_th,u4U):
    
    # Compute v_{0}^{2}
    v0squared = compute_v02( smallbsquared,rhob,dPcold_drho,h,Gamma_th,eps_th )
    
    # Compute a = (1-v_{0}^{2})*(u^{0})^{2} - v_{0}^{2} g^{00}
    a = (1 - v0squared) * u4U[0]**2 + v0squared / alpha**2
    
    # Compute b = 2v_{0}^{2} g^{i0} - 2u^{i}u^{0}(1-v_{0}^{2})
    # and     c = (1-v_{0}^{2})(u^{i})^{2} - v_{0}^{2}g^{ii}
    b = 2 * v0squared * g4UU[flux_dirn+1][0] - 2 * u4U[flux_dirn+1] * u4U[0] * (1 - v0squared)
    c = (1 - v0squared) * u4U[flux_dirn+1]**2 - v0squared * g4UU[flux_dirn+1][flux_dirn+1]
    
    # Compute cplus and cminus, making sure we do
    # not use a negative number in the square root
    detm = b*b - 4*a*c
    detm = sp.sqrt(sp.Rational(1,2)*(detm + nrpyAbs(detm)))
    cplus  = sp.Rational(1,2)*(-b/a + detm/a)
    cminus = sp.Rational(1,2)*(-b/a - detm/a)
            
    return cplus, cminus

<a id='compute_cmax_and_cmin'></a>

## Step 9.c: Computing $c_{\max}$ and $c_{\min}$ \[Back to [top](#toc)\]
$$\label{compute_cmax_and_cmin}$$

We now compute $c_{\max}$ and $c_{\min}$ based on $c_{+,R}$, $c_{-,R}$, $c_{+,L}$, and $c_{-,L}$. The basic idea would be to simply find

\begin{align}
c_{\max} &= \max\left(1,c_{+,R},c_{+,L}\right)\ ,\\
c_{\min} &= \min\left(0,c_{-,R},c_{-,L}\right)\ .
\end{align}

However, because we are working with symbolic expressions, sympy would not be able to discover whether $c_{+,R}>c_{+,L}$ or otherwise (analogously for $c_{-,L,R}$). Therefore, we use the following trick:

\begin{align}
c_{\max} &= \frac{1}{2}\left(c_{+,R} + c_{+,L} + \left|c_{+,R}-c_{+,L}\right|\right) \to \frac{1}{2}\left(c_{\max} + \left|c_{\max}\right|\right)\ ,\\
c_{\min} &= \frac{1}{2}\left(c_{-,R} + c_{-,L} - \left|c_{-,R}-c_{-,L}\right|\right) \to \frac{1}{2}\left(c_{\min} - \left|c_{\min}\right|\right)\ .
\end{align}

In [21]:
def find_cmax_and_cmin(flux_dirn, alpha_face,beta_faceU,gamma_faceDD,
                       smallbsquared_R, smallbsquared_L,
                       rhob_R,dPcold_drho_R,h_R,eps_th_R,u4_RU,
                       rhob_L,dPcold_drho_L,h_L,eps_th_L,u4_LU,
                       Gamma_th):
        
    # Compute g^{\mu\nu} from input
    gamma_faceUU,gamma_facedet = ixp.generic_matrix_inverter3x3(gamma_faceDD)
    AB4m.g4UU_ito_BSSN_or_ADM("ADM",betaU=beta_faceU,alpha=alpha_face,gammaUU=gamma_faceUU)
    g4_faceUU = AB4m.g4UU
    
    # Compute cplus_{R,L} and cminus_{R,L}
    cpR,cmR = compute_cp_and_cm(flux_dirn, g4_faceUU, smallbsquared_R, rhob_R,dPcold_drho_R,h_R,Gamma_th,eps_th_R,u4_RU)
    cpL,cmL = compute_cp_and_cm(flux_dirn, g4_faceUU, smallbsquared_L, rhob_L,dPcold_drho_L,h_L,Gamma_th,eps_th_L,u4_LU)
    
    # Compute cmax = max(0,cpR,cpL)
    cmax =  sp.Rational(1,2)*(cpR+cpL+nrpyAbs(cpR-cpL))
    cmax =  sp.Rational(1,2)*(cmax+nrpyAbs(cmax))
    
    # Compute cmin = max(0,cmR,cmL)
    cmin =  sp.Rational(1,2)*(cmR+cmL-nrpyAbs(cmR-cmL))
    cmin = -sp.Rational(1,2)*(cmin-nrpyAbs(cmin))
    
    return cmax, cmin

<a id='speed_limit_and_u4u'></a>

## Step 9.d: Impose speed limit and compute $u^{\mu}$ \[Back to [top](#toc)\]
$$\label{speed_limit_and_u4u}$$

Here we compute $u^{\mu}$ from $v^{i}$, [after a speed limit has been imposed on $v^{i}$](#impose_speed_limit_output_u0). Then, since we already have access to $u^{0}$ and $v^{i}$, we can compute (see the definition of $v^{i}$ just below equation (8) in [Etienne *et al.*](https://arxiv.org/pdf/1501.07276.pdf)).

$$
\boxed{u^{i} = v^{i} u^{0}}\ .
$$

In [22]:
def compute_u4U_rescaled(alpha_face,beta_faceU,gamma_faceDD, vU):
    
    # Impose speed limit on velocity, compute u^{0} and v^{i}
    rescale_3velocity_vU_and_compute_u0(alpha_face,beta_faceU,gamma_faceDD, vU)
    
    # Set u^{0} = u^{0}_{rescaled}
    rescaledu4U    = ixp.zerorank1(DIM=4)
    rescaledu4U[0] = rescaledu0
    
    # Compute u^{i} = v^{i}_{rescaled} * u^{0}_{rescaled}
    for i in range(DIM):
        rescaledu4U[i+1] = rescaledvU[i] * rescaledu0
        
    return rescaledu4U

<a id='computing_fhll'></a>

## Step 9.e: Computing $F^{\rm HLL}$ \[Back to [top](#toc)\]
$$\label{computing_fhll}$$

$$
\boxed{F^{\rm HLL} = \frac{c^{-}F_{r} + c^{+}F_{l} - c^{+}c^{-}\left(U_{r} - U_{l}\right)}{c^{+} + c^{-}}}\ .
$$

In [23]:
# The code below has been adapted from the HLLE_solver() found in
# Tutorial-GiRaFFE_NRPy_Ccode_library-Stilde-flux.ipynb.
def compute_FHLL(cp,cm,Fr,Fl,Ur,Ul):
    # Compute F^{HLL} = ( cm * Fr + cp * Fl - cm*cp*(Ur - Ul) ) / (cp + cm)
    return ( cm * Fr + cp * Fl - cm*cp*(Ur - Ul) ) / ( cp + cm )

<a id='grmhd_flux'></a>

## Step 9.f: Computing the GRMHD flux \[Back to [top](#toc)\]
$$\label{grmhd_flux}$$

We now compute the GRMHD flux terms for the conservative variables $\left\{\rho_{\star},\tilde{\tau},\tilde{S}_{i}\right\}$, namely

$$
\begin{aligned}
F^{j}_{\rho_{\star}} &= \rho_{\star} v^{j}\ ,\\
F^{j}_{\tilde{\tau}} &= \alpha^{2}\sqrt{\gamma}T^{0j} - \rho_{\star}v^{j}\ ,\\
F_{\tilde{S}_{\ i}}^{j} &= \alpha\sqrt{\gamma}T^{j}_{\ i}\ ,
\end{aligned}
$$

where $j$ represents the flux direction (${\rm flux\_dirn}$) and $i$ is the standard spatial index. After this, we compute the $F^{\rm HLL}$ flux terms for the conservative variables.

In [24]:
def mhdflux(flux_dirn, alpha,betaU,gammaDD,sqrtgamma, Gamma_th,
            rhob_R,P_R,dPcold_drho_R,eps_cold_R,epsilon_R,vRU, smallb4_RU,
            rhob_L,P_L,dPcold_drho_L,eps_cold_L,epsilon_L,vLU, smallb4_LU):

    # Compute g_{\mu\nu}
    AB4m.g4DD_ito_BSSN_or_ADM("ADM",gammaDD=gammaDD,alpha=alpha,betaU=betaU)
    g4DD = AB4m.g4DD

    # Compute b^{2}_{R,L}
    smallbsquared_R = sp.sympify(0)
    smallbsquared_L = sp.sympify(0)
    for mu in range(4):
        for nu in range(4):
            smallbsquared_R += g4DD[mu][nu] * smallb4_RU[mu] * smallb4_RU[nu]
            smallbsquared_L += g4DD[mu][nu] * smallb4_LU[mu] * smallb4_LU[nu]

    # Compute h_{R,L}
    h_R = 1 + epsilon_R + P_R/rhob_R
    h_L = 1 + epsilon_L + P_L/rhob_L
    
    # Compute epsilon_thermal_{R,L}
    eps_th_R = epsilon_R - eps_cold_R
    eps_th_L = epsilon_L - eps_cold_L
    
    # Compute u^{\mu}_{R,L} from v^{i}_{R,L}
    u4_RU = compute_u4U_rescaled(alpha,betaU,gammaDD, vRU)
    u4_LU = compute_u4U_rescaled(alpha,betaU,gammaDD, vLU)
    
    # Compute cmax, cmin
    cmax, cmin = find_cmax_and_cmin(flux_dirn, alpha,betaU,gammaDD,        \
                                    smallbsquared_R, smallbsquared_L,      \
                                    rhob_R,dPcold_drho_R,h_R,eps_th_R,u4_RU, \
                                    rhob_L,dPcold_drho_L,h_L,eps_th_L,u4_LU, \
                                    Gamma_th)
    
    # Compute GRMHD T^{\mu\nu}_{R,L}
    compute_GRMHD_T4UU(alpha,betaU,gammaDD, rhob_R,P_R,epsilon_R,u4_RU, smallb4_RU,smallbsquared_R)
    T4R_UU = T4grmhdUU
    compute_GRMHD_T4UU(alpha,betaU,gammaDD, rhob_L,P_L,epsilon_L,u4_LU, smallb4_LU,smallbsquared_L)
    T4L_UU = T4grmhdUU
    
    # Compute GRMHD T^{\mu}_{\nu}_{R,L} = g_{\nu\sigma} T^{\sigma\mu}
    T4R_UD = ixp.zerorank2(DIM=4)
    T4L_UD = ixp.zerorank2(DIM=4)
    for mu in range(4):
        for nu in range(4):
            for sigma in range(4):
                T4R_UD[mu][nu] += g4DD[nu][sigma] * T4R_UU[sigma][mu]
                T4L_UD[mu][nu] += g4DD[nu][sigma] * T4L_UU[sigma][mu]

    # Compute Conservatives from Primitives (R,L)
    rhostar_R,tau_R,Stilde_RD = primitives_to_conservatives(alpha,betaU,gammaDD,sqrtgamma, rhob_R,P_R,epsilon_R,u4_RU, smallb4_RU)
    rhostar_L,tau_L,Stilde_LD = primitives_to_conservatives(alpha,betaU,gammaDD,sqrtgamma, rhob_L,P_L,epsilon_L,u4_LU, smallb4_LU)

    # Compute F_rhostar (here j represents the direction of the flux):
    # .--------------------------------.
    # | F_{rhostar} = rho_star * v^{j} |
    # .--------------------------------.
    F_rhostar_R = rhostar_R * u4_RU[flux_dirn+1] / u4_RU[0]
    F_rhostar_L = rhostar_L * u4_LU[flux_dirn+1] / u4_LU[0]
    # Compute rho_star_flux using F^{HLL}
    rho_star_flux = compute_FHLL(cmax,cmin,F_rhostar_R,F_rhostar_L,rhostar_R,rhostar_L)

    # Compute F_tau (here j represents the direction of the flux):
    # .---------------------------------------------------------.
    # | F_{tau} = alpha^{2} sqrt(gamma) T^{0j} - rho_star v^{j} |
    # .---------------------------------------------------------.
    F_tau_R = alpha**2 * sqrtgamma * T4R_UU[flux_dirn+1][0] - rhostar_R * u4_RU[flux_dirn+1] / u4_RU[0]
    F_tau_L = alpha**2 * sqrtgamma * T4L_UU[flux_dirn+1][0] - rhostar_L * u4_LU[flux_dirn+1] / u4_LU[0]
    # Compute rho_star_flux using F^{HLL}
    tau_flux = compute_FHLL(cmax,cmin,F_tau_R,F_tau_L,tau_R,tau_L)

    # Compute F_StildeD (here j represents the direction of the flux):
    # .----------------------------------------------.
    # | F_{Stilde_{i}} = alpha sqrt(gamma) T^{j}_{i} |
    # .----------------------------------------------.
    F_Stilde_RD = ixp.zerorank1()
    F_Stilde_LD = ixp.zerorank1()
    for i in range(DIM):
        F_Stilde_RD[i] = alpha * sqrtgamma * T4R_UD[flux_dirn+1][i]
        F_Stilde_LD[i] = alpha * sqrtgamma * T4L_UD[flux_dirn+1][i]
    # Compute StildeD using F^{HLL}
    flux_StildeD = ixp.zerorank1()
    for i in range(DIM):
        flux_StildeD[i] = compute_FHLL(cmax,cmin,F_Stilde_RD[i],F_Stilde_LD[i],Stilde_RD[i],Stilde_LD[i])
        
    return rho_star_flux,tau_flux,flux_StildeD,cmax,cmin

<a id='flux_dirn_i__h'></a>

## Step 9.g: The `flux_dirn_x.h`, `flux_dirn_y.h`, and `flux_dirn_z.h` files \[Back to [top](#toc)\]
$$\label{flux_dirn_i__h}$$

We now write down the `mhdflux.h` file, to replace most of the code in the `mhdflux.C` file from `IllinoisGRMHD`.

In the algorithm below, we assume we have access to the following quantities (as defined in the beginning of the `mhdflux.C` file):

1. $U_{R,L}$: arrays containing primitive variables
1. $\tilde\gamma_{ij}$, $\phi$, $\psi$, $\alpha$, $\beta^{i}$: metric face values. These are stored in the ${\rm FACEVAL}$ array. We also have access to the diagonal components of $\tilde{\gamma}^{ij}$, but it willd be easier for us to just compute $\tilde{\gamma}^{ij}$ from $\tilde{\gamma}_{ij}$, if we need it.


In [25]:
import sympy as sp
import BSSN.ADMBSSN_tofrom_4metric as AB4m

# Step 1: Declare basic ADM variables to be used by IllinoisGRMHD
# Step 1.a: Set spatial dimension to 3
DIM = 3

# Step 1.b: Set up alpha
alpha  = sp.sympify(1) + sp.symbols("FACEVAL[CM_LAPM1]",real=True)
# Step 1.b: Set up beta^{i}
betaU   = ixp.zerorank1()
for i in range(DIM):
    betaU[i] = sp.symbols('FACEVAL[CM_SHIFT'+chr(ord('X')+i)+"]",real=True)

# Step 1.c: Set up \bar\gamma_{ij}
gammabarDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(i,DIM):
        gammabarDD[i][j] = gammabarDD[j][i] = sp.symbols('FACEVAL[CM_GAMMATILDE'+chr(ord('X')+i)+chr(ord('X')+j)+"]",real=True)
        gfslist.append(["GAMMA"+chr(ord('X')+i)+chr(ord('X')+j)])

# Step 1.d: Set up \bar\gamma^{ij}
gammabarUU, gammabardet = ixp.generic_matrix_inverter3x3(gammabarDD)

# Step 1.e: Set up psi^{4} and psi^{-4}
psi   = sp.symbols("FACEVAL[CM_PSI]",real=True)
psi4 = psi**4

# Step 1.e: Set up \gamma_{ij}, \gamma^{ij}, and sqrt(\gamma)
gammaDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        gammaDD[i][j] = psi4 * gammabarDD[i][j]
        
gammaUU, dummydet = ixp.generic_matrix_inverter3x3(gammaDD)
sqrtgamma = psi**6

# Step 3.a.ii: Declare needed variables, named according to IllinoisGRMHD
Gamma_th = sp.symbols("Gamma_th",real=True)
rhob_L,P_L,P_cold_L,dPcold_drho_L,eps_cold_L = sp.symbols("Ul[RHOB] Ul[PRESSURE] P_coldl dPcold_drhol eps_coldl",real=True)
rhob_R,P_R,P_cold_R,dPcold_drho_R,eps_cold_R = sp.symbols("Ur[RHOB] Ur[PRESSURE] P_coldr dPcold_drhor eps_coldr",real=True)

# Step 3.a.iii: Compute epsilon
epsilon_L = eps_cold_L + (P_L - P_cold_L)/(rhob_L * (Gamma_th - 1))
epsilon_R = eps_cold_R + (P_R - P_cold_R)/(rhob_R * (Gamma_th - 1))

# Step 3.a.iv: Compute u^{\mu} from v^{i}
vLU = [sp.symbols("Ul[VX]",real=True),sp.symbols("Ul[VY]",real=True),sp.symbols("Ul[VZ]",real=True)]
vRU = [sp.symbols("Ur[VX]",real=True),sp.symbols("Ur[VY]",real=True),sp.symbols("Ur[VZ]",real=True)]

smallb4_LU = [sp.symbols("smallbl[SMALLBT]",real=True),sp.symbols("smallbl[SMALLBX]",real=True), \
              sp.symbols("smallbl[SMALLBY]",real=True),sp.symbols("smallbl[SMALLBZ]",real=True)]

smallb4_RU = [sp.symbols("smallbr[SMALLBT]",real=True),sp.symbols("smallbr[SMALLBX]",real=True), \
              sp.symbols("smallbr[SMALLBY]",real=True),sp.symbols("smallbr[SMALLBZ]",real=True)]

varlist = ["rho_star_flux","tau_flux",         \
           "st_x_flux","st_y_flux","st_z_flux",\
           "cmax","cmin"]

for i in range(DIM):
    rhoflux,tauflux,stildefluxD,cmax,cmin = mhdflux(int(i), alpha,betaU,gammaDD,sqrtgamma, Gamma_th,
                                                    rhob_R,P_R,dPcold_drho_R,eps_cold_R,epsilon_R,vRU, smallb4_RU,
                                                    rhob_L,P_L,dPcold_drho_L,eps_cold_L,epsilon_L,vLU, smallb4_LU)
    exprlist = [rhoflux,tauflux,stildefluxD[0],stildefluxD[1],stildefluxD[2],cmax,cmin]
    
    string = outputC(exprlist,varlist,filename="returnstring", params="outCverbose=False")

    # Step 7.e: Replace pow(blah, 2) with (blah)*(blah) and double with CCTK_REAL
    string2 = re.sub('pow\(([^,]+), 2\)', '(\\1)*(\\1)', string); string = string2
    string2 = re.sub('double','CCTK_REAL',string); string = string2
    
    filename="flux_dirn_"+chr(ord('x')+i)+".h"
    filepath = os.path.join(NRPy_headers_dir_path,filename)
    NRPy_IGM_write_to_file(filepath,filename,string)

Just generated the file: ../src/NRPy_generated_headers/flux_dirn_x.h
Just generated the file: ../src/NRPy_generated_headers/flux_dirn_y.h
Just generated the file: ../src/NRPy_generated_headers/flux_dirn_z.h


<a id='latex_pdf_output'></a>

# Step n: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-IllinoisGRMHD__NRPyfied_IGM_expressions.pdf](Tutorial-IllinoisGRMHD__NRPyfied_IGM_expressions.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means).

In [26]:
latex_nrpy_style_path = os.path.join(nrpy_dir_path,"latex_nrpy_style.tplx")
!jupyter nbconvert --to latex --template $latex_nrpy_style_path Tutorial-IllinoisGRMHD__NRPyfied_IGM_expressions.ipynb
!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__NRPyfied_IGM_expressions.tex
!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__NRPyfied_IGM_expressions.tex
!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__NRPyfied_IGM_expressions.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-IllinoisGRMHD__NRPyfied_IGM_expressions.ipynb to latex
[NbConvertApp] Writing 207101 bytes to Tutorial-IllinoisGRMHD__NRPyfied_IGM_expressions.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
